<a href="https://colab.research.google.com/github/Edimosphy/Symptom_MediCare/blob/main/Symptom_Medicare_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Install streamlit
!pip install streamlit

In [4]:
#Import the necessary librabies
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#####For this demo, ten (10) Physical Symptoms will be used for the health simualted

In [5]:
data = {
    'Disease': ['Malaria', 'Malaria', 'Malaria',
                'Typhoid', 'Typhoid', 'Typhoid',
                'HIV/AIDS', 'HIV/AIDS', 'HIV/AIDS'],
    'Fever': ['High', 'Medium', 'High',
              'High', 'High', 'Medium',
              'Medium', 'Low', 'Low'],
    'Fatigue': ['Very High', 'High', 'High',
                'High', 'Medium', 'Low',
                'Very High', 'High', 'High'],
    'Headache': ['High', 'Medium', 'High',
                 'Very High (Heaviness)', 'High', 'Medium',
                 'Low', 'Medium', 'Medium'],
    'Vomiting': ['Yes', 'Yes', 'Yes',
                 'Yes', 'Yes', 'No',
                 'Yes', 'Yes', 'Yes'],
    'Skin Rash': ['Mild', 'Mild', 'None',
                  'Rose spots', 'Mild', 'None',
                  'High', 'High', 'Medium'],
    'Muscle Joint Pain': ['Yes', 'No', 'Medium',
                   'No', 'Yes', 'Medium',
                   'Yes', 'Yes', 'Yes'],
    'Weight Loss': ['Moderate', 'Mild', 'Severe',
                    'Mild', 'Mild', 'Moderate',
                    'Severe', 'Severe', 'Severe'],
    'Diarrhea': ['No', 'Yes', 'No',
                 'Yes', 'No', 'Yes',
                 'Yes', 'Yes', 'Yes'],
    'Night Sweats': ['Yes', 'Yes', 'No',
                 'No', 'No', 'No',
                 'Yes', 'Yes', 'Yes'],
    'Lymph Node Swelling': ['No', 'No', 'No',
                 'No', 'No', 'No',
                 'Yes', 'Yes', 'High']


}

df = pd.DataFrame(data)
df.head()

,Disease,Fever,Fatigue,Headache,Vomiting,Skin Rash,Muscle Joint Pain,Weight Loss,Diarrhea,Night Sweats,Lymph Node Swelling
0,Malaria,High,Very High,High,Yes,Mild,Yes,Moderate,No,Yes,No
1,Malaria,Medium,High,Medium,Yes,Mild,No,Mild,Yes,Yes,No
2,Malaria,High,High,High,Yes,None,Medium,Severe,No,No,No
3,Typhoid,High,High,Very High (Heaviness),Yes,Rose spots,No,Mild,Yes,No,No
4,Typhoid,High,Medium,High,Yes,Mild,Yes,Mild,No,No,No


In [7]:
# Symptom input options by mapping the unique option
symptom_option_mapping  = {
    'Fever': {'a': 'High', 'b': 'Low', 'c': 'Medium'},
    'Fatigue': {'a': 'High', 'b': 'Low', 'c': 'Medium', 'd': 'Very High'},
    'Headache': {'a': 'High', 'b': 'Low', 'c': 'Medium', 'd': 'Very High (Heaviness)'},
    'Vomiting': {'a': 'Yes', 'b': 'No'},
    'Skin Rash': {'a': 'Mild', 'b': 'None', 'c': 'High'},
    'Muscle Joint Pain': {'a': 'Yes', 'b': 'No', 'c': 'Medium'},
    'Weight Loss': {'a': 'Mild', 'b': 'Moderate', 'c': 'Severe'},
    'Diarrhea': {'a': 'Yes', 'b': 'No'},
    'Night Sweats': {'a': 'Yes', 'b': 'No'},
    'Lymph Node Swelling': {'a': 'No', 'b': 'Yes', 'c': 'High'}
}


In [13]:
# Use Naive Bayes prediction
def predict_disease(df, user_symptoms):
    disease_probs = {}
    total_count = len(df)
    diseases = df['Disease'].unique()

    for disease in diseases:
        sub_df = df[df['Disease'] == disease]
        prior = len(sub_df) / total_count
        likelihood = 1.0

        for symptom, value in user_symptoms.items():
            match_count = len(sub_df[sub_df[symptom] == value])
            symptom_prob = (match_count + 1) / (len(sub_df) + len(df[symptom].unique())) if len(sub_df) else 0.0
            likelihood *= symptom_prob

        disease_probs[disease] = prior * likelihood

    total_prob = sum(disease_probs.values())

    if total_prob == 0:
        return "No Match Found", {}
    else:
        for disease in disease_probs:
            disease_probs[disease] = (disease_probs[disease] / total_prob) * 100

        predicted = max(disease_probs, key=disease_probs.get)
        return predicted, disease_probs

In [14]:
# Plot the probabilities for visual display of the disease
def plot_advanced_distribution(probabilities):
    sns.set(style="whitegrid")
    diseases = list(probabilities.keys())
    values = list(probabilities.values())
    colors = sns.color_palette("coolwarm", len(diseases))

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = sns.barplot(x=diseases, y=values, palette=colors, ax=ax, hue=diseases, legend=False)

    ax.set_title("Symptom Medicare Disease Prediction", fontsize=16, fontweight='bold')
    ax.set_xlabel("Disease", fontsize=12)
    ax.set_ylabel("Probability (%)", fontsize=12)
    ax.set_ylim(0, 100)

    for bar in bars.patches:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 2,
                f'{height:.1f}%', ha='center', fontsize=11, fontweight='semibold')

    return fig # Return the figure object

In [15]:
# Streamlit App Layout
st.title("Symptom Medicare Disease Predictor")
st.write("Please select the symptoms you are experiencing.")

user_symptoms = {}
# Create select boxes for each symptom
for symptom, options in symptom_option_mapping.items():
    # Get the list of values for the selectbox
    option_values = list(options.values())
    selected_value = st.selectbox(f"Select {symptom}", option_values)
    # Store the selected value in the user_symptoms dictionary
    user_symptoms[symptom] = selected_value

if st.button("Predict Disease"):
    prediction, probs = predict_disease(df, user_symptoms)

    if prediction == "No Match Found":
        st.write("No potential disease matches found for the given symptoms in the training data.")
    else:
        st.write(f"\nBased on your symptoms, the most likely disease is: **{prediction}**")
        if probs:
            fig = plot_advanced_distribution(probs)
            st.pyplot(fig)

2025-05-17 16:56:46.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.117 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-17 16:56:46.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 16:56:46.127 Thread 'MainThread': mi